### Most variables have been included; code cleaning and validation are still required.

In [ ]:
import os
import time
import json
from openai import AzureOpenAI
from dotenv import load_dotenv

# Load environment variables
load_dotenv()

# Initialize Azure OpenAI client
client = AzureOpenAI(
    #api_key=os.getenv("AZURE_OPENAI_API_KEY"),
    api_version="2023-07-01-preview",
    azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT")
)

# Prompt user input
user_input = input("Describe the TLOF (Landing Surface) in natural language:\n")

# Function definition
function_def = [
    {
        "name": "generate_landing_surface_layout",
        #"description": "Generate a structured TLOF (Touchdown and Lift-Off Area) JSON layout with visual and dimensional properties. Defaults will be applied where values are missing.",
        "description": "Generate a structured JSON layout for a TLOF (Touchdown and Lift-Off Area), including visual, geometric, lighting, and safety parameters. The layout should support customization for shape (Polygon, Circle, Rectangle), aircraft type, size, elevation, markings (solid or dashed), and landing marker (capital letter 'H' or 'V'). If any field is not specified, apply reasonable defaults. Note: 'markingType' accepts only 'solid' or 'dashed'. 'landingMarker' accepts only 'H' or 'V'.",
        "parameters": {
            "type": "object",
            "properties": {
                "aircraft": {"type": "string"},
                "surface_length": {"type": "number"},
                "surface_width": {"type": "number"},
                "surface_height": {"type": "number"},
                "sides": {"type": "number"},
                "diameter": {"type": "number"},
                "marking_thickness": {"type": "number"},
                "rotation": {"type": "number"},
                "transparency": {"type": "number"},
                "elevation": {"type": "number"},
                "shape_type": {
                    "type": "string",
                    "enum": ["Circle", "Polygon", "Rectangle"]
                },
                "position": {
                    "type": "array",
                    "items": {"type": "number"},
                    "minItems": 2,
                    "maxItems": 2
                },
                "lightColor": {"type": "string"},
                "lightScale": {"type": "number"},
                "lightDistance": {"type": "number"},
                "lightRadius": {"type": "number"},
                "lightHeight": {"type": "number"},
                "markingType": {"type": "string"},
                "dashLength": {
                    "type": "number",
                    "description": "Length of each dash in dashed marking. Only used when markingType is 'dashed'."
                },
                "dashDistance": {
                    "type": "number",
                    "description": "Distance between dashes in dashed marking. Only used when markingType is 'dashed'."
                },
                "markingColor": {"type": "string"},
                
                "landingMarker": {
                    "type": "string",
                    "description": "Capital letter for the landing marker. Accepts only 'H' or 'V'."
                },
                "markerScale": {
                    "type": "number",
                    "description": "Scale of the landing marker. Typical range: 0.5 to 50."
                },
                "markerThickness": {
                    "type": "number",
                    "description": "Extrusion thickness of the landing marker (in meters)."
                },
                "letterThickness": {
                    "type": "number",
                    "description": "Line thickness of the marker letter."
                },
                "markerRotation": {
                    "type": "number",
                    "description": "Rotation angle of the landing marker (0–359°)."
                },
                "markerColor": {
                    "type": "string",
                    "description": "Color of the landing marker. Accepts only: white, yellow, blue, red, green, black, purple, orange, gray, brown.",
                    "enum": ["white", "yellow", "blue", "red", "green", "black", "purple", "orange", "gray", "brown"]
                },
                "tdpcCategory": {
                    "type": "boolean",
                    "description": "Enable TDPC (Touchdown Positioning Circle) marking. This should be set to true only if any TDPC-related property (type, scale, thickness, extrusion, rotation, or color) is explicitly provided or modified. Otherwise, keep it false by default."
                },
                "tdpcType": {
                    "type": "string",
                    "description": "Shape type of the TDPC. Recommended: 'circle', 'cross', or 'square'.",
                    "enum": ["circle", "cross", "square"]
                },
                "tdpcScale": {
                    "type": "number",
                    "description": "Scale or radius of the TDPC. Valid range: 0.5 to 50.",
                    "minimum": 0.5,
                    "maximum": 50
                },
                "tdpcThickness": {
                    "type": "number",
                    "description": "Line thickness of the TDPC. Valid range: 0.1 to 50.",
                    "minimum": 0.1,
                    "maximum": 50
                },
                "tdpcExtrusion": {
                    "type": "number",
                    "description": "Extrusion thickness of the TDPC for 3D appearance. Valid range: 0 to 1.",
                    "minimum": 0,
                    "maximum": 1
                },
                "tdpcRotation": {
                    "type": "number",
                    "description": "Rotation angle of the TDPC in degrees (0–359).",
                    "minimum": 0,
                    "maximum": 359
                },
                "tdpcColor": {
                    "type": "string",
                    "description": "Color of the TDPC. Must be one of the supported colors.",
                    "enum": ["white", "yellow", "blue", "red", "green", "black", "purple", "orange", "gray", "brown"]
                },
                "safetyAreaCategory": {
                    "type": "boolean",
                    "description": "Enable safety area configuration. This is automatically set to true if any safety area parameters are customized (like dValue, multiplier, or offsetDistance)."
                },
                "safetyAreaType": {
                    "type": "string",
                    "description": "Defines the method used to calculate the safety area around the TLOF. Use 'multiplier' to scale based on dValue, or 'offset' to use a fixed distance.",
                    "enum": ["offset", "multiplier"]
                },
                "dValue": {
                    "type": "number",
                    "description": "D-Value for safety area sizing. Required when safetyAreaType is 'multiplier'. Typical range: 1 to 20 meters."
                },
                "multiplier": {
                    "type": "number",
                    "description": "Multiplication factor applied to dValue to calculate the safety area. Used only with safetyAreaType set to 'multiplier'. Typical range: 0.5 to 20."
                },
                "offsetDistance": {
                    "type": "number",
                    "description": "Fixed offset distance (in meters) around the TLOF used to define the safety area. Used only when safetyAreaType is 'offset'. Typical range: 1 to 50 meters."
                },
                "safetyNetCategory": {
                    "type": "boolean",
                    "description": "Enable safety net rendering. Used to define safety net attributes like color, texture, height, and transparency."
                },
                "curveAngle": {
                    "type": "number",
                    "description": "Curve angle of the safety net boundary in degrees. Default is 45.",
                    "minimum": 0,
                    "maximum": 90
                },
                "netHeight": {
                    "type": "number",
                    "description": "Height of the safety net in meters. Default is 15.",
                    "minimum": 0,
                    "maximum": 100
                },
                "safetyNetTransparency": {
                    "type": "number",
                    "description": "Transparency of the safety net (0 to 1).",
                    "minimum": 0,
                    "maximum": 1
                }
            },
            "required": [
                "aircraft", "surface_length", "surface_width", "surface_height",
                "sides", "diameter", "marking_thickness", "rotation", "transparency",
                "shape_type", "position"
            ]
        }
    }
]

# Get AI response
response = client.chat.completions.create(
    model=os.getenv("AZURE_OPENAI_MODEL_NAME"),
    messages=[{"role": "user", "content": user_input}],
    functions=function_def,
    function_call={"name": "generate_landing_surface_layout"}
)

# Parse function arguments
function_args = json.loads(response.choices[0].message.function_call.arguments)

# Defaults
function_args["elevation"] = function_args.get("elevation") or 20.0
allowed_light_colors = {"white", "yellow", "blue", "red", "green", "black", "purple", "orange", "gray", "brown"}
input_color = function_args.get("lightColor", "white").lower()
if input_color not in allowed_light_colors:
    print(f"⚠️ Unsupported lightColor '{input_color}', defaulting to 'white'.")
    input_color = "white"

# Light category logic
default_light_values = {
    "lightColor": "white",
    "lightScale": 1,
    "lightDistance": 1,
    "lightRadius": 0.3,
    "lightHeight": 0.2
}
light_category_enabled = any(
    function_args.get(k) is not None and function_args.get(k) != v
    for k, v in default_light_values.items()
)

# Validate landing marker
allowed_markers = {"H", "V"}
marker_value = function_args.get("landingMarker", "H").upper()
if marker_value not in allowed_markers:
    print(f"⚠️ Unsupported landingMarker '{marker_value}', defaulting to 'H'.")
    marker_value = "H"

default_marking_values = {
    "markingType": "dashed",
    "markingColor": "white",
    "marking_thickness": 1.5,
    "dashLength": 2.2,
    "dashDistance": 2.5
}
markings_category_enabled = any(
    str(function_args.get(k, default_marking_values[k])).lower() != str(default_marking_values[k]).lower()
    for k in default_marking_values
)

# Validate markingType
allowed_marking_types = {"solid", "dashed"}
marking_type = function_args.get("markingType", "dashed").lower()
if marking_type not in allowed_marking_types:
    print(f"⚠️ Unsupported markingType '{marking_type}', defaulting to 'dashed'.")
    marking_type = "dashed"

# Set dash values conditionally
if marking_type == "dashed":
    dash_length = max(0.5, min(function_args.get("dashLength", 2.2), 5))
    dash_distance = max(0.5, min(function_args.get("dashDistance", 2.5), 5))
else:
    # Solid line → set minimum placeholder values
    dash_length = 0.5
    dash_distance = 0.5



default_marker_values = {
    "landingMarker": "H",
    "markerScale": 1,
    "markerThickness": 0.02,
    "letterThickness": 0.5,
    "markerRotation": 0,
    "markerColor": "white"
}
landing_marker_category_enabled = any(
    str(function_args.get(k, default_marker_values[k])).lower() != str(default_marker_values[k]).lower()
    for k in default_marker_values
)

# Landing marker value bounds
marker_scale = max(0.5, min(function_args.get("markerScale", 1), 50))
marker_thickness = max(0, min(function_args.get("markerThickness", 0.02), 5))
letter_thickness = max(0, min(function_args.get("letterThickness", 0.5), 50))
marker_rotation = max(0, min(function_args.get("markerRotation", 0), 359))

#TDPC Marking
tdpc_keys = {
    "tdpcType", "tdpcScale", "tdpcThickness",
    "tdpcExtrusion", "tdpcRotation", "tdpcColor"
}
tdpc_category_enabled = any(k in function_args and function_args[k] != default_marker_values.get(k) for k in tdpc_keys)

# Safety Area
default_safety_area_values = {
    "safetyAreaType": "offset",
    "dValue": 10,
    "multiplier": 1.5,
    "offsetDistance": 1.5    
}
# Enable safety area only if any parameter differs from defaults
safety_area_category_enabled = any(function_args.get(k, default_safety_area_values[k]) != default_safety_area_values[k] for k in default_safety_area_values)

# Safety Net
default_safety_net_values = {
    "curveAngle": 45,
    "netHeight": 15,
    "safetyNetTransparency": 0.5
}

# Enable safety net only if any parameter differs from defaults
safety_net_category_enabled = any(
    function_args.get(k, default_safety_net_values[k]) != default_safety_net_values[k]
    for k in default_safety_net_values
)


# ID
tlof_id = f"TLOF-{int(time.time() * 1000)}"

# Final JSON
tlof_json = {
    "TLOF": [
        {
            "position": function_args["position"],  # [Lng, Lat]
            "dimensions": {
                "unit": "m",
                "sides": max(3, min(function_args["sides"], 70)),  # Sides
                "width": max(1, min(function_args["surface_width"], 30)),
                #"dValue": max(1, min(function_args.get("dValue", 10), 20)),  # D-Value (m)
                "height": max(0.01, min(function_args["surface_height"], 5)),  # Thickness
                "length": max(1, min(function_args["surface_length"], 30)),
                "aircraft": function_args["aircraft"],
                "diameter": max(1.0, min(function_args["diameter"], 200.0)),  # Diameter
                "rotation": max(0, min(function_args["rotation"], 359)),  # Rotation
                
                "isVisible": True,
                "layerName": "Praveen_TLOF",
                "shapeType": function_args["shape_type"],
                "baseHeight": max(0.0, min(function_args["elevation"], 1000.0)),  # Elevation
                #"multiplier": max(0.5, min(function_args.get("multiplier", 1.5), 20)),  # Multiplication Factor                
                "transparency": max(0, min(function_args["transparency"], 1)),  # Transparency
                "scaleCategory": False,
                "textureScaleU": 1,  # Texture: #808080
                "textureScaleV": 1,
                "vertipadColor": "#808080",  # Texture: #808080
                #"safetyAreaType": "multiplier",  # With Safety Area_D-Value Multiplier
                #"safetyNetColor": "#FF0000",  # Safety Net Color: #FF0000
                "safetyNetScaleU": 1,
                "safetyNetScaleV": 1,
                "aircraftCategory": False,

                # Marking
                "markingsCategory": markings_category_enabled,
                "markingType": marking_type,  # Marking: Solid/Dashed
                "markingColor": function_args.get("markingColor", "white"), #Marking Color: White
                "markingThickness": max(0.1, min(function_args["marking_thickness"], 1.5)),  #Perimeter Marking Extrusion
                "dashLength": dash_length,
                "dashDistance": dash_distance,
                
                #Landing Marker
                "landingMarkerCategory": landing_marker_category_enabled,
                "landingMarker": marker_value,  # Landing Marker Type (H or V)
                #"markerScale": max(0.5, min(function_args.get("markerScale", 1), 50)),  # Landing Marker : Scalar Multiplier
                #"markerThickness": max(0, min(function_args.get("markerThickness", 0.02), 1)),  #Landing Marker : Extrusion Thickness
                #"letterThickness": max(0, min(function_args.get("letterThickness", 0.5), 50)),  # Landing Marker Thickness
                #"markerRotation": max(0, min(function_args.get("markerRotation", 0), 359)),  # Landing Marker Rotation
                "markerScale": marker_scale,
                "markerThickness": marker_thickness,
                "letterThickness": letter_thickness,
                "markerRotation": marker_rotation,
                "markerColor": function_args.get("markerColor", "white"),  #Landing Marker Color
                
                #TDPC Marking
                "tdpcCategory": tdpc_category_enabled,
                "tdpcType": "circle",
                "tdpcScale": max(0.5, min(function_args.get("tdpcScale", 5), 50)),  # TDPC Radius
                "tdpcThickness": max(0.1, min(function_args.get("tdpcThickness", 0.5), 50)),  # Line Thickness
                "tdpcExtrusion": max(0, min(function_args.get("tdpcExtrusion", 0.02), 1)),  # Extrusion
                "tdpcRotation": max(0, min(function_args.get("tdpcRotation", 0), 359)),  # Rotation
                "tdpcColor": function_args.get("tdpcColor", "white"),  # TDPC Color

                #Lighting
                "lightCategory": light_category_enabled,
                "lightColor": input_color,  # Lighting: White
                "lightScale": max(-20, min(function_args.get("lightScale") or 1, 100)),  # Perimeter Edge Offset
                "lightDistance": max(0.5, min(function_args.get("lightDistance") or 1, 50)),  # Distance between Lights
                "lightRadius": max(0.1, min(function_args.get("lightRadius") or 0.3, 1)),  # Lights Base Radius
                "lightHeight": max(0.1, min(function_args.get("lightHeight") or 0.2, 0.25)),  # Light Extrusion Height
                
                # Safety Area
                "safetyAreaCategory": safety_area_category_enabled,
                "safetyAreaType": function_args.get("safetyAreaType", "offset"),  # "offset" or "multiplier"
                "dValue": max(1, min(function_args.get("dValue", 10), 20)),        # D-Value (m)
                "multiplier": max(0.5, min(function_args.get("multiplier", 1.5), 20)),  # Multiplication Factor
                "offsetDistance": max(0.5, min(function_args.get("offsetDistance", 1.5), 20)), 

                # Safety Net
                "safetyNetCategory": safety_net_category_enabled,
                "curveAngle": max(0, min(function_args.get("curveAngle", 10), 90)),
                "netHeight": max(0, min(function_args.get("netHeight", 10), 100)),
                "safetyNetTransparency": max(0.1, min(function_args.get("lightRadius") or 0.3, 1))
            }
        }
    ]
}


# Print output
print(json.dumps(tlof_json, indent=2))


Describe the TLOF (Landing Surface) in natural language:
 Generate a TLOF with safety net angle 45°, height 80, and transparency 0.3, reflecting these in the design.


{
  "TLOF": [
    {
      "position": [
        0,
        0,
        0
      ],
      "dimensions": {
        "unit": "m",
        "sides": 4,
        "width": 30,
        "height": 0.5,
        "length": 30,
        "aircraft": "helicopter",
        "diameter": 1.0,
        "rotation": 0,
        "isVisible": true,
        "layerName": "Praveen_TLOF",
        "shapeType": "Polygon",
        "baseHeight": 20.0,
        "transparency": 1,
        "scaleCategory": false,
        "textureScaleU": 1,
        "textureScaleV": 1,
        "vertipadColor": "#808080",
        "safetyNetScaleU": 1,
        "safetyNetScaleV": 1,
        "aircraftCategory": false,
        "markingsCategory": true,
        "markingType": "solid",
        "markingColor": "white",
        "markingThickness": 0.15,
        "dashLength": 0.5,
        "dashDistance": 0.5,
        "landingMarkerCategory": true,
        "landingMarker": "H",
        "markerScale": 10,
        "markerThickness": 0.5,
        "letterThickn